## Initial imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os, pickle
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
from chython import smiles

## Model rebuilding

The hyperparameters of the models are stored in *trials.all* and *trials.best* file after optimization. While these tables are not difficult to read from a human perspective, rebuilding any model manually is not the most interesting process. To help with tat, we provide the utiliity in the rebuilder module to rebuild any model reported in the abovementioned files.

In [2]:
# getting the best regression model for 
best_regr = pd.read_table("lambda/Model_Epipi_regr/trials.best", sep=" ").iloc[0]

from doptools.cli.rebuilder import rebuild_from_file

pipeline, trial = rebuild_from_file("lambda/circus", "lambda/Model_Epipi_regr", best_regr["trial"])
pipeline

Pipeline(steps=[('descriptor_calculator', ChythonCircus(upper=5)),
                ('variance', VarianceThreshold()),
                ('model',
                 SVR(C=2466.9567970597163, coef0=-0.5036670063859954,
                     gamma='auto'))])

In [3]:
photoswitches = pd.read_table("../examples/photoswitches.csv", sep=",", index_col=0)
data_lambda = photoswitches[pd.notnull(photoswitches['E isomer pi-pi* wavelength in nm'])]
lambda_mols = [smiles(s) for s in data_lambda.SMILES]
[m.canonicalize() for m in lambda_mols]
[m.clean2d() for m in lambda_mols]
y = np.array(data_lambda["E isomer pi-pi* wavelength in nm"])
pipeline.fit(lambda_mols, y)

Pipeline(steps=[('descriptor_calculator', ChythonCircus(upper=5)),
                ('variance', VarianceThreshold()),
                ('model',
                 SVR(C=2466.9567970597163, coef0=-0.5036670063859954,
                     gamma='auto'))])

The *pipeline* object now contains the trained model, from the descriptor generation to numerical prediction. Next, let's give it another azo-dye to predict the absorption maximum. The actual value of $\lambda_{max}$ for it in 462.

In [4]:
ext_mol = smiles("C(C)N(CC)c1ccc(cc1)N=Nc2ccc(cc2)C(=O)C")
ext_mol.clean2d()

ext_pred = pipeline.predict([ext_mol])
ext_pred

array([417.48361672])

## Model interpretation

The regression models built in DOPtools with fragment descriptors can be interpreted by the built-in ColorAtom methodology. 

In ColorAtom, the fragment weights are calcualted as partial derivatives of predictions, and all atoms participating is these fragments are assigned these weights. After summarizing these wights over all the framgnets, the atomic contribution can be then visualizaed. Below is an example of such interpretation on the previously introduced molecule.

In [5]:
from doptools import ColorAtom

ca = ColorAtom()
ca.set_pipeline(pipeline) 
# it is necessary to indicate what pipeline, containing 
# the fragmentor, preprocessing and model, is used 
# for interpretation

ca.output_html(ext_mol)


The results are output as an HTML table containing SVG images for the molecule. The color indicates the contribution - red for negative (towards lower value), green for positive (towards higher value). The intensity of color indicates the magnitue of the contribution, the brighter the color, the more the atom contributes. 

To compare the contributions of different molecules, the user may visualize the colorbar with the indication of the scale of the contributions. To do that, the parameter "colorbar" need to be given.

In [6]:
ca.output_html(ext_mol, colorbar=True)

In [13]:
ext_mol2 = smiles("C(C)N(CC)c1ccc(cc1)N=Nc2ccc(nc2)C(=O)O")
ext_mol2.clean2d()
ca.output_html(ext_mol2, colorbar=True)

The color scheme of the visualization may also be changed, either by using the predefined colormaps in Matplotlib, or defining your own.

In [9]:
from doptools import ColorAtom

ca = ColorAtom()
ca.set_pipeline(pipeline) 
# it is necessary to indicate what pipeline, containing 
# the fragmentor, preprocessing and model, is used 
# for interpretation

ca.output_html(ext_mol)

In [8]:
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

# defining a new colormap, going from blue for negative to red for positive
RWB = LinearSegmentedColormap.from_list("RWB", 
        ["#0571b0","#92c5de","#f7f7f7","#f4a582","#ca0020"])

ca2 = ColorAtom(colormap=RWB)
ca2.set_pipeline(pipeline) 
ca2.output_html(ext_mol, colorbar=True)

## Classification case

In [17]:
best_class = pd.read_table("bbb/Model_chyline/trials.best", sep=" ").iloc[0]

from doptools.cli.rebuilder import rebuild_from_file

pipeline_class, trial = rebuild_from_file("bbb/descs/chyline", "bbb/Model_chyline", best_class["trial"])
pipeline_class

Pipeline(steps=[('descriptor_calculator', ChythonLinear(lower=2, upper=5)),
                ('scaler', MinMaxScaler()), ('variance', VarianceThreshold()),
                ('model',
                 RandomForestClassifier(max_depth=10, max_features=None,
                                        max_samples=0.3, n_estimators=200))])

In [18]:
bbb = pd.read_excel("bbb/BBB_dataset.xlsx")
bbb_mols = [smiles(s) for s in bbb.mol]
[m.canonicalize() for m in bbb_mols]
[m.clean2d() for m in bbb_mols]
y = np.array(bbb["Active"])
pipeline_class.fit(bbb_mols, y)

Pipeline(steps=[('descriptor_calculator', ChythonLinear(lower=2, upper=5)),
                ('scaler', MinMaxScaler()), ('variance', VarianceThreshold()),
                ('model',
                 RandomForestClassifier(max_depth=10, max_features=None,
                                        max_samples=0.3, n_estimators=200))])

In [21]:
ca_class = ColorAtom()
ca_class.set_pipeline(pipeline_class) 
# SMILES of the test molecule here is 
# CC(C)(OC(=O)[C@@H](C)N)Cc1ccc(cc1)Cl

ca_class.output_html(bbb_mols[15], colorbar=True)